In [1]:
import pandas as pd
import numpy as np
import requests, bs4
import re, os
import matplotlib.pyplot as plt
import statsapi
from datetime import date



In [2]:
today = date.today()
today = str(today)[-2:]
today

url = f'https://www.baseballmusings.com/cgi-bin/CurStreak.py?EndDate=07%2F{today}%2F2022'
resp = requests.get(url)

with open('test_mlb_longesthitstreak', 'wb') as f:
    f.write(resp.content)

df_streaks = pd.read_html('test_mlb_longesthitstreak')
df1 = pd.DataFrame(df_streaks[1])
df1 = df1.loc[(df1['Player'] != 'Player')]
df1

,Player,Games,At Bats,Runs,Hits,HR,RBI,BB,K,BA,OBA,Slug%,Last Game Date
0,Austin Riley,15,62,14,27,8,15,3,13,.435,.462,.903,2022-07-23
1,Carson Kelly,12,41,9,17,3,7,3,7,.415,.455,.732,2022-07-23
2,Freddie Freeman,11,41,12,22,5,11,6,1,.537,.592,1.049,2022-07-23
3,Isiah Kiner-Falefa,11,39,2,12,0,9,0,5,.308,.317,.385,2022-07-23
4,Amed Rosario,10,40,8,18,0,8,2,6,.450,.476,.625,2022-07-23
5,Charlie Blackmon,10,44,7,17,2,12,1,4,.386,.391,.659,2022-07-23
6,Trea Turner,10,43,12,15,4,10,1,6,.349,.364,.674,2022-07-23
7,Adam Frazier,9,34,6,14,1,3,1,4,.412,.417,.529,2022-07-23
8,Alejandro Kirk,8,33,7,13,1,5,2,4,.394,.417,.515,2022-07-23
9,Brendan Rodgers,8,33,3,11,1,7,2,10,.333,.351,.515,2022-07-23


In [3]:
def get_todays_hitstreak_info(df):
    todays_info = []
    for name in df['Player']:
        team_id = statsapi.lookup_player(name)[0]['currentTeam']['id']
        full_team_name = statsapi.lookup_team(team_id)[0]['name']
        team_abbrv = statsapi.lookup_team(team_id)[0]['teamCode']
        team_name = statsapi.lookup_team(team_id)[0]['teamName']
        player_info = (name, full_team_name, team_id, team_abbrv.upper(), team_name)
        todays_info.append(player_info)
    print(pd.DataFrame(todays_info, columns = ['Player', 'Full Team Name', 'Team ID', 'Team Abbrv', 'Mascot']).sort_values(by = 'Team Abbrv'))
        
get_todays_hitstreak_info(df1)

                   Player         Full Team Name  Team ID Team Abbrv  \
22           Luis Rengifo     Los Angeles Angels      108        ANA   
1            Carson Kelly   Arizona Diamondbacks      109        ARI   
0            Austin Riley         Atlanta Braves      144        ATL   
39        Travis D'Arnaud         Atlanta Braves      144        ATL   
10             Matt Olson         Atlanta Braves      144        ATL   
34            Jorge Mateo      Baltimore Orioles      110        BAL   
27            Ramon Urias      Baltimore Orioles      110        BAL   
37      Enrique Hernandez         Boston Red Sox      111        BOS   
41          Rob Refsnyder         Boston Red Sox      111        BOS   
13          Andrew Vaughn      Chicago White Sox      145        CHA   
31      Christopher Morel           Chicago Cubs      112        CHN   
29         Donovan Solano        Cincinnati Reds      113        CIN   
20            Steven Kwan    Cleveland Guardians      114       

In [8]:
team_dict = {
    108: ('LAA', 'Angels', '#GoHalos'),
    109: ('ARI', 'D-backs', '#Dbacks'),
    110: ('BAL', 'Orioles', '#Birdland'),
    111: ('BOS', 'Red Sox', '#DirtyWater'),
    112: ('CHC', 'Cubs', '#ItsDifferentHere'),
    113: ('CIN', 'Reds', '#ATOBTTR'),
    114: ('CLE', 'Guardians', '#ForTheLand'),
    115: ('COL', 'Rockies', '#Rockies'),
    116: ('DET', 'Tigers', '#DetroitRoots'),
    117: ('HOU', 'Astros', '#LevelUp'),
    118: ('KC', 'Royals', '#TogetherRoyal'),
    119: ('LAD', 'Dodgers', '#AlwaysLA'),
    120: ('WSH', 'Nationals', '#NATITUDE'),
    121: ('NYM', 'Mets', '#LGM'),
    133: ('OAK', 'Athletics', '#DrumTogether'),
    134: ('PIT', 'Pirates', '#LetsGoBucs'),
    135: ('SD', 'Padres', '#TimeToShine'),
    136: ('SEA', 'Mariners', '#SeaUsRise'),
    137: ('SF', 'Giants', '#SFGameUp'),
    138: ('STL', 'Cardinals', '#STLCards'),
    139: ('TB', 'Rays', '#RaysUp'),
    140: ('TEX', 'Rangers', '#StraightUpTX'),
    141: ('TOR', 'Blue Jays', '#NextLevel'),
    142: ('MIN', 'Twins', '#MNTwins'),
    143: ('PHI', 'Phillies', '#RingTheBell'),
    144: ('ATL', 'Braves', '#ForTheA'),
    145: ('CWS', 'White Sox', '#ChangeTheGame'),
    146: ('MIA', 'Marlins', '#MakeItMiami'),
    147: ('NYY', 'Yankees', '#RepBX'),
    158: ('MIL', 'Brewers', '#ThisIsMyCrew')
}

In [ ]:
api.

In [ ]:
pd.DataFrame([('Austin Riley', 144, 'Atlanta Braves', 'atl', 'Braves'), ('Francisco Lindor', 121, 'New York Mets', 'nyn', 'Mets'), ('Corey Seager', 140, 'Texas Rangers', 'tex', 'Rangers'), ('Carson Kelly', 109, 'Arizona Diamondbacks', 'ari', 'D-backs'), ('Brandon Drury', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Freddie Freeman', 119, 'Los Angeles Dodgers', 'lan', 'Dodgers'), ('Leody Taveras', 140, 'Texas Rangers', 'tex', 'Rangers'), ('Charlie Blackmon', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Jose Abreu', 145, 'Chicago White Sox', 'cha', 'White Sox'), ("Ke'Bryan Hayes", 134, 'Pittsburgh Pirates', 'pit', 'Pirates'), ('Amed Rosario', 114, 'Cleveland Guardians', 'cle', 'Guardians'), ('Adam Frazier', 136, 'Seattle Mariners', 'sea', 'Mariners'), ('Seiya Suzuki', 112, 'Chicago Cubs', 'chn', 'Cubs'), ('Trea Turner', 119, 'Los Angeles Dodgers', 'lan', 'Dodgers'), ('Jonathan India', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Nick Castellanos', 143, 'Philadelphia Phillies', 'phi', 'Phillies'), ('Isiah Kiner-Falefa', 147, 'New York Yankees', 'nya', 'Yankees'), ('Alex Kirilloff', 142, 'Minnesota Twins', 'min', 'Twins'), ('Garrett Hampson', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Lourdes Gurriel', 141, 'Toronto Blue Jays', 'tor', 'Blue Jays'), ('Alejandro Kirk', 141, 'Toronto Blue Jays', 'tor', 'Blue Jays'), ('Brendan Rodgers', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Tim Anderson', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Matt Olson', 144, 'Atlanta Braves', 'atl', 'Braves'), ('Alec Bohm', 143, 'Philadelphia Phillies', 'phi', 'Phillies'), ('Austin Nola', 135, 'San Diego Padres', 'sdn', 'Padres'), ('Lars Nootbaar', 138, 'St. Louis Cardinals', 'sln', 'Cardinals'), ('Nelson Cruz', 120, 'Washington Nationals', 'was', 'Nationals'), ('Luke Williams', 146, 'Miami Marlins', 'mia', 'Marlins'), ('Yoan Moncada', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Ryan Jeffers', 142, 'Minnesota Twins', 'min', 'Twins'), ('Harold Castro', 116, 'Detroit Tigers', 'det', 'Tigers'), ('Taylor Walls', 139, 'Tampa Bay Rays', 'tba', 'Rays'), ('Skye Bolt', 133, 'Oakland Athletics', 'oak', 'Athletics'), ('Ryan McMahon', 115, 'Colorado Rockies', 'col', 'Rockies'), ('C.J. Cron', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Yasmani Grandal', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Enrique Hernandez', 111, 'Boston Red Sox', 'bos', 'Red Sox'), ('Joey Votto', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Nicky Lopez', 118, 'Kansas City Royals', 'kca', 'Royals')])

,0,1,2,3,4
0,Austin Riley,144,Atlanta Braves,atl,Braves
1,Francisco Lindor,121,New York Mets,nyn,Mets
2,Corey Seager,140,Texas Rangers,tex,Rangers
3,Carson Kelly,109,Arizona Diamondbacks,ari,D-backs
4,Brandon Drury,113,Cincinnati Reds,cin,Reds
5,Freddie Freeman,119,Los Angeles Dodgers,lan,Dodgers
6,Leody Taveras,140,Texas Rangers,tex,Rangers
7,Charlie Blackmon,115,Colorado Rockies,col,Rockies
8,Jose Abreu,145,Chicago White Sox,cha,White Sox
9,Ke'Bryan Hayes,134,Pittsburgh Pirates,pit,Pirates
